In [16]:
import pandas as pd
import numpy as np
import traceback
import datetime
import sys
import locale
from sklearn import preprocessing

In [2]:
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_columns', None)

# Importar Datasets

In [3]:
def import_data():
    path = "Datasets/"#diretório dos datasets
    
    # --------- as linhas abaixo servem para gerar o nome dos arquivos automaticamente
    prefix = "ABONOP_"#prefixo comun a todos os arquivos
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']#meses presentes
    years = ["2017", "2018", "2019", "2020"]#anos presentes
    ext = ".csv"#extensão dos arquivos
    
    # ------------------------
    
    columns = ["Nome", "CPF", "Descrição do cargo emprego", "Nível de Escolaridade", "Denominação do órgão de atuação",
              "UF da UPAG de vinculação", "Denominação unidade organizacional", "UF da Residência",
              "Cidade da residência", "Situação servidor", "Quantidade de anos no Serviço público",
              "Quantidade de meses no Serviço público","Ano/Mês inicial DO ABONO DE PERMANENCIA","VAL"] #colunas presentes
    
    date_columns = ["Ano/Mês inicial DO ABONO DE PERMANENCIA"]#colunas que devem ser transformadas em datas, porém no dataset estão como inteiros
    float_columns = {"VAL":np.float64}#coluna númerica
    
    datasets = []#esta variavel vai ser usada como uma lista de DataFrames importados
    sep = ";"#separador utilizado nos arquivos
    decimal = "."#valores decimais são separados por virgula
    try:
        name = ""
        for i in years:# esse loop vai importar os datasets ano a ano
            for j in months:# esse loop vai importar os datasets mês a mês
                name = path + prefix + j + i + ext# concatena as informações do arquivo para formar seu nome

                df = pd.read_csv(name, sep =sep, #low_memory = False,
                                names = columns, header = 0,
                                #warn_bad_lines = True
                                encoding = 'cp1252', index_col=False)#ver documentação do pandas
                date = datetime.date(int(i), int(j), 1)#cria uma data do tipo 1/j/i
                df["DATE"] = [date for k in range(df.shape[0])]#adiciona uma coluna com a data
                datasets.append(df)
    except Exception as e:#ocorreu algum exceção
        traceback.print_exc()#printa a pilha de execução
        print("Erro na planilha: {}".format(name))#printa a plainlha que ocorreu o erro
    finally:
        if len(datasets) > 0:
            datasets = pd.concat(datasets)#concatena os datasets que foram importados com sucesso
            datasets.drop_duplicates(subset = columns, inplace = True)#tira as colunas duplicadas, não considera a coluna DATE
            
            datasets.reset_index(inplace = True)
            datasets.drop(columns = ["index"], inplace = True)
            
        return datasets

In [4]:
def import_dataset(name, sep = ",", columns = []):
    df = pd.read_csv(name, sep =sep, #low_memory = False,
                    names = columns, header = 0,
                    #warn_bad_lines = True
                   index_col=False)#ver documentação do pandas
    return df

In [5]:
abono_dataset = import_data()

In [6]:
cidades_dataset = import_dataset("Datasets/municipios.csv", sep = ";", columns = ["ConcatUF+Mun", "IBGE", "IBGE7", "UF", "Município", 
                                               "Região", "População 2010", "Porte", "Capital"])

C:\Users\samue\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


# Retirar espaços em branco em excesso e alguns ajustes

In [7]:
def clean_spaces(df, text_columns, number_columns):
    #retira os espaços em branco e coloca todos os caracteres em caixa baixa
    df = df.applymap(
        lambda x: " ".join( x.split() ).lower() if isinstance(x, str) else x, 
        na_action = 'ignore')
    #troca todas as ',' por '.'
    df[number_columns] = df[number_columns].applymap(
        lambda x: ".".join(x.split(",")) if isinstance(x, str) else x, 
        na_action = 'ignore')
    
    return df

In [8]:
text_columns = ["Nome", "CPF", "Descrição do cargo emprego", "Nível de Escolaridade", "Denominação do órgão de atuação",
              "UF da UPAG de vinculação", "Denominação unidade organizacional", "UF da Residência",
              "Cidade da residência", "Situação servidor"]
number_columns = ["Quantidade de anos no Serviço público","Quantidade de meses no Serviço público",
                  "Ano/Mês inicial DO ABONO DE PERMANENCIA","VAL"]
abono_dataset = clean_spaces(abono_dataset, text_columns, number_columns)

In [9]:
dtypes = {
    "Quantidade de anos no Serviço público": int,
    "Quantidade de meses no Serviço público": int,
    "VAL": np.float64
}
abono_dataset = abono_dataset.astype(dtypes)

# Corrigir coluna Cidade da residência

In [10]:
def distancia_hamming(string1, string2):
    """
    Calcula a distancia de hamming entre as strings de entrada
    """
    #código de https://stackoverflow.com/questions/54172831/hamming-distance-between-two-strings-in-python
    return sum(c1 != c2 for c1, c2 in zip(string1, string2))

In [11]:
def corrige_cidade(cidades_do_estado , cidade):
    """
    Essa função corrige a cidade dada por uma na lista cidades_do_estado
    A função de distancia de hamming é utilizada como critério para escolha
    O algoritmo escolhe o elemento com menor distância de hamming da lista cidades_do_estado
    com cidade
    """
    if cidade in cidades_do_estado:#verifica se a cidade já existe na lista
        return cidade
    else:#se não ela vai ser processada
        cid_min = ""#inicializa a cidade com distancia de hamming minima
        distancia_min = 999999#inicializa a minima distancia de hamming encontrada
        for cid in cidades_do_estado:#percorre todas as cidades do estado
            distancia = distancia_hamming(cid, cidade)#calcula a distancia entre a cidade atual e a cidade informada
            if distancia < distancia_min:
                #se essa distancia for menor que a minima até agora
                #então é feita uma substituição
                cid_min = cid
                distancia_min = distancia
        return cid_min#retorna a cidade com distancia minima encontrada

In [12]:
estados = set(cidades_dataset["UF"].to_list())
cidades_por_estado = dict()
for estado in estados:#organiza um diciionário de cidades por estado
    cds = cidades_dataset.loc[cidades_dataset["UF"] == estado]["Município"].to_list()
    cidades_por_estado[estado] = list(map(lambda x: x.lower(), cds))

In [13]:
#realiza correção das cidades
idx_cidade = 9#indice da coluna cidade
idx_uf = 8#indice da coluna uf
mudancas = []#esse array vai salvando as correçoes de cada linha
cache = dict()#dicionario de cache para poupar processamento
for row in abono_dataset.itertuples():
    uf = row[idx_uf].upper()#transforma em caixa alta pois as chaves em cidades_por_estado estão em caixa alta, 
    if uf in cidades_por_estado:# se a uf da linha existe no diciionario de cidades_por_estado
        
        cd_desconhecida = row[idx_cidade]
        cidade = ""
        if cd_desconhecida in cache:#verifica se a cidade já foi processada
            cidade = cache[cd_desconhecida]#recupera o resultado
        else:#se não foi
            cidades = cidades_por_estado[uf]#obtem as cidades do estado daquela linha
            cidade = corrige_cidade(cidades, cd_desconhecida)#corrige a cidade
            cache[cd_desconhecida] = cidade#salva o resultado na cache
        mudancas.append(cidade)#adiciona na lista de mudancas
    else:# se não existe não faz nenhum processamento
        mudancas.append(row[idx_cidade])
        #print(row[idx_cidade], cidade)
        
abono_dataset["Cidade da residência"] = pd.Series(data = mudancas)

# Corrigir coluna Nível de Escolaridade

In [14]:
#realiza correção do nível de escolaridade
idx_escolaridade = 4#indice da coluna Nível de Escolaridade

mud_escolaridade = {
    "segundo grau incompleto": "ensino medio incompleto",
    "4a. serie do primeiro grau completa": "ensino fundamental incompleto",
    "primeiro grau incomp.-ate a 4a.serie incomp.": "ensino fundamental incompleto"
}
mudancas =[]
for row in abono_dataset.itertuples():
    if row[idx_escolaridade] in mud_escolaridade:
        mudancas.append(mud_escolaridade[row[idx_escolaridade]])
    else:
         mudancas.append(row[idx_escolaridade])
abono_dataset["Nível de Escolaridade"] = pd.Series(data = mudancas)

# Corrigir coluna Ano/Mês inicial DO ABONO DE PERMANENCIA	

In [ ]:
#converte a coluna para datetime.date

In [53]:
def converte_int_date(df, coluna):
    """Transforma a coluna de inteiros em datas"""
    df[coluna] = df[coluna].apply(lambda x: datetime.date(x//100, x%100, 1) if isinstance(x, int) else x)#coloca o dia como 1 por padrão

In [54]:
converte_int_date(abono_dataset, "Ano/Mês inicial DO ABONO DE PERMANENCIA")

# Codificação de campos strings para inteiros

### UF da Residência

In [30]:
le = preprocessing.LabelEncoder()
le.fit(abono_dataset["UF da Residência"])
abono_dataset["UF da Residência"] = le.transform(abono_dataset["UF da Residência"])

### Situação servidor

In [31]:
le = preprocessing.LabelEncoder()
le.fit(abono_dataset["Situação servidor"])
abono_dataset["Situação servidor"] = le.transform(abono_dataset["Situação servidor"])

### Cidade da residência

In [32]:
le = preprocessing.LabelEncoder()
le.fit(abono_dataset["Cidade da residência"])
abono_dataset["Cidade da residência"] = le.transform(abono_dataset["Cidade da residência"])

### Nível de Escolaridade

In [33]:
le = preprocessing.LabelEncoder()
le.fit(abono_dataset["Nível de Escolaridade"])
abono_dataset["Nível de Escolaridade"] = le.transform(abono_dataset["Nível de Escolaridade"])

# Adiciona coluna meses com abono

In [75]:
def sub_date(data_init = None, data_fim = None):
    meses = 0
    if (data_init != None) and (data_fim != None):
        anos = data_fim.year - data_init.year#diferença de anos
        meses = (data_fim.month - data_init.month + 1) + (anos*12)#quantos meses se passou da data final + quantidade de meses da passagem de anos
    return meses

In [77]:
idx_init = 13#indice Ano/Mês inicial DO ABONO DE PERMANENCIA
idx_date = 15#indice DATE
meses =[]
for row in abono_dataset.itertuples():
    meses.append(sub_date( data_init = row[idx_init], data_fim = row[idx_date]))
    
abono_dataset["Meses com abono"] = pd.Series(data = meses)

# Testes

In [78]:
abono_dataset

,Nome,CPF,Descrição do cargo emprego,Nível de Escolaridade,Denominação do órgão de atuação,UF da UPAG de vinculação,Denominação unidade organizacional,UF da Residência,Cidade da residência,Situação servidor,Quantidade de anos no Serviço público,Quantidade de meses no Serviço público,Ano/Mês inicial DO ABONO DE PERMANENCIA,VAL,DATE,Meses com abono
0,jose alberto fontenele magalhaes,***287981**,tecnico de contabilidade,6,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas-cgap,7,410,4,57,5,2004-05-01,590.84,2017-01-01,153
1,dalva galvao pinho,***311601**,datilografo,4,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas-cgap,7,410,4,50,11,2004-05-01,576.69,2017-01-01,153
2,cicero joaquim dos santos filho,***204574**,auditor fiscal federal agropecuario,6,"minist.da agricultura,pecuaria e abast.",pe,sup fed agric pec abast est pernambuco,16,632,4,42,6,2003-12-01,2241.38,2017-01-01,158
3,marlete de fatima lopes de albuquerque,***782921**,agente administrativo,4,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas-cgap,7,410,4,41,5,2008-05-01,546.11,2017-01-01,105
4,paulo elisio de faria,***402121**,agente administrativo,6,"minist.da agricultura,pecuaria e abast.",df,coord geral adm de pessoas-cgap,7,410,4,40,5,2010-01-01,558.32,2017-01-01,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739586,elias bastos de oliveira,***497697**,tecnico de contabilidade,4,comando da marinha,rj,comando do primeiro distrito naval,19,1762,4,33,7,2006-07-01,526.96,2020-12-01,174
2739587,manoel de oliveira,***943857**,aux operac de serv diversos,3,comando da marinha,rj,arsenal de marinha do rio de janeiro,19,325,4,32,4,2014-02-01,520.95,2020-12-01,83
2739588,eli manoel teodoro,***416027**,aux operac de serv diversos,4,comando da marinha,rj,arsenal de marinha do rio de janeiro,19,325,4,30,10,2010-07-01,515.30,2020-12-01,126
2739589,monica fraga de barros silva,***010877**,analista em ciencia e tecnologia,6,comando da marinha,rj,centro de analises de sistemas navais,19,2194,4,10,0,2015-10-01,1653.16,2020-12-01,63
